In [1]:
import json
import pandas as pd
import evaluate

path = 'results/bula_all.jsonl'

mini_paciente = pd.read_csv('datasets/mini_paciente.csv')

list_json = []

with open(path, 'r') as file:
    for line in file:
        line_json = json.loads(line)
        line_json['full_topic'] = mini_paciente.loc[mini_paciente['id'] == line_json['id'], 'full_topic'].values
        list_json.append(line_json)

from bula_process.models import Mistral, Rag, Llama3
model = Mistral()
rag = Rag()

def compute_bleu(y_true, y_pred):
    metric = evaluate.load('bleu')
    metric.add_batch(predictions=y_pred, references=y_true)
    report = metric.compute()
    bleu = report['bleu'] * 100
    return bleu

def compute_bertscore(y_pred, y_true):
    metric = evaluate.load('bertscore')
    reference_batch = [[y_true]]
    sys_batch = [y_pred]
    metric.add_batch(predictions=sys_batch, references=reference_batch)
    report = metric.compute(model_type="distilbert-base-uncased")
    return report

def make_question_rag(id):
    data = list_json[id]
    context = rag.retrieve(data['query'])
    question_prompt = f"[INST]Answer the question: [/INST] Context: {context} \n {data['query']} \nAnswer: "
    answer_model = model.inference(question_prompt)

    gold_choice = ''


    for idx, choice in enumerate(data['choices']):
        if idx == int(data['gold']):
            gold_choice = choice


    print(f"Resposta do modelo:")

    print(answer_model)

    print(f"Resposta certa:")
    print(gold_choice)
    list_a = [[data['query'], answer_model]]
    list_b = [gold_choice]
    bert_score = compute_bertscore(answer_model, gold_choice)
    print('\n')
    print(f"BERT Score: {bert_score}")
    bleu = compute_bleu(list_a, list_b)
    print(f"BLEU metric: {bleu}")


def make_question_vanilla(id):
    data = list_json[id]
    question_prompt = f"[INST]Answer the question: [/INST] " + data['query'] + "\nAnswer: "
    answer_model = model.inference(question_prompt)

    gold_choice = ''
    for idx, choice in enumerate(data['choices']):
        if idx == int(data['gold']):
            gold_choice = choice

    print(data['query'])
    print(f"Resposta do modelo:")

    print(answer_model)
    print(f"Resposta certa:")
    print(gold_choice)
    list_a = [[data['query'], answer_model]]
    list_b = [gold_choice]
    bert_score = compute_bertscore(answer_model, gold_choice)
    print('\n')

    print(f"BERT Score: {bert_score}")
    bleu = compute_bleu(list_a, list_b)
    print(f"BLEU metric: {bleu}")

/home/navarro/miniconda3/envs/python310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.22it/s]
/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
make_question_rag(109)

Resposta do modelo:
    Based on the provided text, the possible side effects or adverse reactions of using Neo Mistatin creme include irritation and sensitivity, including sensations of burning and itching. However, the text does not mention any specific information about severe side
Resposta certa:
irritação e sensibilidade, incluindo sensação de queimação e coceira


/home/navarro/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(




BERT Score: {'precision': [0.6267098188400269], 'recall': [0.629981279373169], 'f1': [0.6283412575721741], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.40.2)'}
BLEU metric: 0.0


In [3]:
make_question_vanilla(109)

Quais são os sintomas de efeitos colaterais que podem ocorrer com o uso de Neo Mistatin creme?
Resposta do modelo:
    The Neo-Mistatin cream is a topical medication used to treat various skin infections. Like all medications, it can cause side effects, although not everyone who uses it will experience them. Some common side effects of Neo-M
Resposta certa:
irritação e sensibilidade, incluindo sensação de queimação e coceira


BERT Score: {'precision': [0.5973389744758606], 'recall': [0.608060359954834], 'f1': [0.6026520133018494], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.40.2)'}
BLEU metric: 0.0


In [4]:
make_question_rag(95)

Resposta do modelo:
    One common side effect (>1/10) that can occur with the use of Oxaliplatina and is related to hematological function is leucopenia.
Resposta certa:
Anemia


BERT Score: {'precision': [0.6504451036453247], 'recall': [0.7298442125320435], 'f1': [0.6878610253334045], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.40.2)'}
BLEU metric: 0.0


In [5]:
make_question_vanilla(95)

Qual é a reação muito comum (>1/10) que pode ocorrer com o uso de Oxaliplatina e é relacionada à função hematológica?
Resposta do modelo:
    A common reaction (more than 1 in 10 people) that can occur with the use of Oxaliplatin and is related to its hematologic function is Neutropenia. Neutropenia is a condition
Resposta certa:
Anemia


BERT Score: {'precision': [0.6361693739891052], 'recall': [0.7469609975814819], 'f1': [0.6871278882026672], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.40.2)'}
BLEU metric: 0.0


In [6]:
make_question_rag(213)

Resposta do modelo:
    The maximum daily dose for long-term treatment with Cloridrato de verapamil should not exceed 480 mg. However, higher doses may be used for short-term treatment. The dose should be adjusted individually according to
Resposta certa:
480 mg


BERT Score: {'precision': [0.6046256422996521], 'recall': [0.8787660598754883], 'f1': [0.7163643836975098], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.40.2)'}
BLEU metric: 0.0


In [7]:
make_question_vanilla(213)

Qual é a dose máxima diária recomendada para o tratamento longo com o Cloridrato de verapamil?
Resposta do modelo:
    The maximum daily dose for long-term treatment with Verapamil Hydrochloride varies depending on the individual's condition and response. The usual dose ranges from 120 to 180 mg/day, given in divided
Resposta certa:
480 mg


BERT Score: {'precision': [0.5868179202079773], 'recall': [0.7835495471954346], 'f1': [0.6710622310638428], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.40.2)'}
BLEU metric: 0.0
